In [ ]:
import beamtest_analysis_helper as helper
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted
import hist
from tqdm.notebook import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use('CMS')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
columns_to_read = ['evt', 'board', 'row', 'col', 'toa', 'tot', 'cal']
names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2p01_BAR_5", "ET2_EPIR_Pair4"]

high_voltages = [250, 260, 210, 260]
offsets = [15, 10, 10, 15]

chip_figtitles = [
    f"(Trigger) EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"(DUT1) Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"(DUT2) Barcelona 5 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"(Reference) EPIR Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"
]

dut1_id = 1
dut2_id = 2
ref_id = 3

In [ ]:
files = natsorted(glob('DESYFeb2024_Run_3*_feather/*feather'))
files[:3]

In [ ]:
last_evt = 0
dataframes = []

pixel_buffer = 2
single_pixel_row = 7
single_pixel_col = 4

for idx, ifile in enumerate(tqdm(files)):

    tmp_df = pd.read_feather(ifile, columns=columns_to_read)

    if tmp_df.empty:
        continue

    subset_df = tmp_df.loc[ ((tmp_df['board'] == 0) | (tmp_df['board'] == dut1_id) | (tmp_df['board'] == ref_id))
                           & ( (tmp_df['row'] >= single_pixel_row-pixel_buffer) & (tmp_df['row'] <= single_pixel_row+pixel_buffer) )
                           & ( (tmp_df['col'] >= single_pixel_col-pixel_buffer) & (tmp_df['col'] <= single_pixel_col+pixel_buffer) )
                           ]
    del tmp_df

    event_board_counts = subset_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
    event_selection_col = (event_board_counts[0] == 1) & (event_board_counts[dut1_id] == 1) & (event_board_counts[ref_id] == 1)

    isolated_df = subset_df.loc[subset_df['evt'].isin(event_board_counts[event_selection_col].index)]
    isolated_df.reset_index(inplace=True, drop=True)

    new_df = isolated_df.loc[(isolated_df['row'] == single_pixel_row) & (isolated_df['col'] == single_pixel_col)]

    event_board_counts = new_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
    event_selection_col = (event_board_counts[0] == 1) & (event_board_counts[dut1_id] == 1) & (event_board_counts[ref_id] == 1)

    isolated_df = new_df.loc[new_df['evt'].isin(event_board_counts[event_selection_col].index)]
    isolated_df.reset_index(inplace=True, drop=True)

    isolated_df['evt'] = isolated_df.groupby('evt').ngroup().astype('uint64')

    if idx > 0:
        isolated_df['evt'] += np.uint64(last_evt)

    last_evt += np.uint64(isolated_df['evt'].nunique())


    dataframes.append(isolated_df)
    del isolated_df, new_df, event_board_counts, event_selection_col

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

In [ ]:
h_inclusive = helper.return_hist(df, chipNames=names, chipLabels=[0,1,2,3], hist_bins=[100, 128, 128])

for iboard in [0,1,2,3]:
    helper.plot_1d_TDC_histograms(h_inclusive, names[iboard], names[iboard], chip_figtitles[iboard], save=False,
                                tag="tdc_Selection", fig_tag=", TDC cuts", slide_friendly=True)

del h_inclusive

In [ ]:
## Selecting good hits
tdc_cuts = {}
for idx in [0,1,3]:
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    if idx == 0:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  100, 500, 100, 250]
    elif idx == ref_id:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  0, 1100, 100, 200]
    else:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  0, 1100, 0, 600]

filtered_df = helper.tdc_event_selection(df, tdc_cuts_dict=tdc_cuts, select_by_hit=False)
del df

In [ ]:
filtered_df.info()

In [ ]:
h_after_tdc = helper.return_hist(filtered_df, chipNames=names, chipLabels=[0,1,2,3], hist_bins=[100, 128, 128])

for iboard in [0,1,2,3]:
    helper.plot_1d_TDC_histograms(h_after_tdc, names[iboard], names[iboard], chip_figtitles[iboard], save=False,
                                tag="tdc_Selection", fig_tag=", TDC cuts", slide_friendly=True)

del h_after_tdc

In [ ]:
plot_filterd_df = filtered_df.loc[(filtered_df['board'] == 0) | (filtered_df['board'] == ref_id)]
helper.plot_TDC_correlation_scatter_matrix(input_df=plot_filterd_df, chip_names=names, single_hit=False, colinear=False, save=False)
del plot_filterd_df

In [ ]:
x = filtered_df.loc[filtered_df['board'] == 0]['toa'].reset_index(drop=True)
y = filtered_df.loc[filtered_df['board'] == ref_id]['toa'].reset_index(drop=True)

params = np.polyfit(x, y, 1)

h_test = hist.Hist(
    hist.axis.Regular(128, 0, 1024, name='toa_0', label='toa_0'),
    hist.axis.Regular(128, 0, 1024, name=f'toa_{ref_id}', label=f'toa_{ref_id}'),
)
h_test.fill(x.values, y.values)

hep.hist2dplot(h_test, norm=colors.LogNorm())

# calculate the trendline
trendpoly = np.poly1d(params)

# plot the trend line
plt.plot(x.values, trendpoly(x.values), 'r-')

In [ ]:
x = filtered_df.loc[filtered_df['board'] == 0]['toa'].reset_index(drop=True)
y = filtered_df.loc[filtered_df['board'] == dut1_id]['toa'].reset_index(drop=True)

params = np.polyfit(x, y, 1)

h_test = hist.Hist(
    hist.axis.Regular(128, 0, 1024, name='toa_0', label='toa_0'),
    hist.axis.Regular(128, 0, 1024, name=f'toa_{dut1_id}', label=f'toa_{dut1_id}'),
)
h_test.fill(x.values, y.values)

hep.hist2dplot(h_test, norm=colors.LogNorm())

# calculate the trendline
trendpoly = np.poly1d(params)

# plot the trend line
plt.plot(x.values, trendpoly(x.values), 'r-')

### Correlation cut (distance cut) 1. trig + ref and 2. trig + dut

In [ ]:
x = filtered_df.loc[filtered_df['board'] == 0]['toa'].reset_index(drop=True)
y = filtered_df.loc[filtered_df['board'] == ref_id]['toa'].reset_index(drop=True)

trig_ref_params = np.polyfit(x, y, 1)
trig_ref_distance = (x*trig_ref_params[0] - y + trig_ref_params[1])/(np.sqrt(trig_ref_params[0]**2 + 1))

x = filtered_df.loc[filtered_df['board'] == 0]['toa'].reset_index(drop=True)
y = filtered_df.loc[filtered_df['board'] == dut1_id]['toa'].reset_index(drop=True)

trig_dut_params = np.polyfit(x, y, 1)
trig_dut_distance = (x*trig_dut_params[0] - y + trig_dut_params[1])/(np.sqrt(trig_dut_params[0]**2 + 1))

In [ ]:
pivot_table = filtered_df.pivot(index=["evt"], columns=["board"], values=["row", "col", "toa", "tot", "cal"])
pivot_table = pivot_table.reset_index()
pivot_table = pivot_table[(trig_ref_distance.abs() < 3*np.std(trig_ref_distance)) & (trig_dut_distance.abs() < 3*np.std(trig_dut_distance))]
pivot_table = pivot_table.reset_index(drop=True)
pivot_table

In [ ]:
board_to_analyze = [0,1,3]
iteration = 100
sampling_fraction = 0.75
counter = 0
sum_arr = defaultdict(float)
sum_square_arr = defaultdict(float)
final_dict = defaultdict(list)
each_res = defaultdict(list)

for iloop in tqdm(range(iteration)):

    tdc_filtered_df = pivot_table

    n = int(sampling_fraction*tdc_filtered_df.shape[0])
    indices = np.random.choice(tdc_filtered_df['evt'].unique(), n, replace=False)
    tdc_filtered_df = tdc_filtered_df.loc[tdc_filtered_df['evt'].isin(indices)]

    if tdc_filtered_df.shape[0] < iteration/(3.*(1-sampling_fraction)):
        print('Warning!! Sampling size is too small. Skipping this track')
        break

    d = {
        'evt': tdc_filtered_df['evt'].unique(),
    }

    for idx in board_to_analyze:
        bins = 3125/tdc_filtered_df['cal'][idx].mean()
        d[f'toa_b{str(idx)}'] = 12.5 - tdc_filtered_df['toa'][idx] * bins
        d[f'tot_b{str(idx)}'] = (2*tdc_filtered_df['tot'][idx] - np.floor(tdc_filtered_df['tot'][idx]/32)) * bins

    df_in_time = pd.DataFrame(data=d)
    del d, tdc_filtered_df

    if(len(board_to_analyze)==3):
        corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, 5, 3, board_list=board_to_analyze)
    elif(len(board_to_analyze)==4):
        corr_toas = helper.four_board_iterative_timewalk_correction(df_in_time, 5, 3)
    else:
        print("You have less than 3 boards to analyze")
        break

    diffs = {}
    for board_a in board_to_analyze:
        for board_b in board_to_analyze:
            if board_b <= board_a:
                continue
            name = f"{board_a}{board_b}"
            diffs[name] = np.asarray(corr_toas[f'toa_b{board_a}'] - corr_toas[f'toa_b{board_b}'])

    try:
        fit_params = {}
        for key in diffs.keys():
            params = helper.fwhm_based_on_gaussian_mixture_model(diffs[key], n_components=2, each_component=False, plotting=False)
            fit_params[key] = float(params[0]/2.355)
            # gmm_sigmas.append(params[1])
        del params, diffs, corr_toas

        resolutions = helper.return_resolution_three_board_fromFWHM(fit_params, var=list(fit_params.keys()), board_list=board_to_analyze)

        if any(np.isnan(val) for key, val in resolutions.items()):
            print('fit results is not good, skipping this iteration')
            continue

        for key in resolutions.keys():
            each_res[key].append(resolutions[key])
            sum_arr[key] += resolutions[key]
            sum_square_arr[key] += resolutions[key]**2

        counter += 1

    except Exception as inst:
        print(inst)
        del diffs, corr_toas

if counter != 0:
    for idx in board_to_analyze:
        final_dict[f'row{idx}'].append(pivot_table['row'][idx].unique()[0])
        final_dict[f'col{idx}'].append(pivot_table['col'][idx].unique()[0])

    for key in sum_arr.keys():
        mean = sum_arr[key]/counter
        std = np.sqrt((1/(counter-1))*(sum_square_arr[key]-counter*(mean**2)))
        final_dict[f'res{key}'].append(mean)
        final_dict[f'err{key}'].append(std)

In [ ]:
each_res

In [ ]:
plt.hist(each_res[3], bins=15)

In [ ]:
final_dict